In [ ]:
# !pip install datetime sklearn

In [ ]:
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('../data/processed/engineered_data.csv')

# Calculate RFM metrics
snapshot_date = datetime(2023, 12, 31)
rfm = df.groupby('CustomerId').agg({
    'TransactionStartTime': lambda x: (snapshot_date - pd.to_datetime(x).max()).days,
    'TransactionId': 'count',
    'Amount': 'sum'
}).reset_index()
rfm.columns = ['CustomerId', 'Recency', 'Frequency', 'Monetary']

# Clustering
X = StandardScaler().fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])
kmeans = KMeans(n_clusters=3, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(X)

# Assume cluster 0 is high-risk
high_risk_cluster = rfm.groupby('Cluster')['Monetary'].mean().idxmin()
rfm['is_high_risk'] = (rfm['Cluster'] == high_risk_cluster).astype(int)

df = df.merge(rfm[['CustomerId', 'is_high_risk']], on='CustomerId')
df.to_csv('../data/processed/labeled_data.csv', index=False)
